In [1]:
import faiss
import json
import numpy as np
import torch

from transformers import AutoModel, AutoTokenizer


class DataLoader:

    def __init__(self, path):
        self.path = path


    def load_data(self):
        with open(self.path, 'r') as file:
            data = json.load(file)
        return data


    def save_data(self, data ,save_path):
        with open(save_path, 'w') as file:
            json.dump(data, file)


class TransformerEncoder:

    """Elements of the TransformerEncoder class were influence by a blog post from Txus Bach
       https://www.codegram.com/blog/finding-similar-documents-with-transformers/"""


    def __init__(self, text_document, model_type):
        self.data = text_document
        self.tokenizer = AutoTokenizer.from_pretrained(model_type)
        self.model = AutoModel.from_pretrained(model_type)

    
    def text_encoder(self, text):
        text_tokens = self.tokenizer(text, return_tensors='pt')
        embed = self.model(**text_tokens)[0].detach().squeeze()
        return torch.mean(embed, dim=0)

    
    def search(self, query, index, k=5):
        encoded_query = self.text_encoder(query).unsqueeze(dim=0).detach().numpy()
        I, D = index.search(encoded_query, k)
        return [(self.data[i], j) for i, j in zip(D[0], I[0])]



if __name__ == '__main__':
    data = DataLoader('../data/sentences.json').load_data()

    encoder = TransformerEncoder(data, 'distilbert-base-uncased')
    vectors = [encoder.text_encoder(d) for d in data]
    index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
    index.add_with_ids(np.array([vec.numpy() for vec in vectors]), np.array(range(0, len(data))))

    query = 'How to prevent the spread of viral infections?'
    result = encoder.search(query, index, k=5)
    print(result)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[('A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.', 60.5831), ('As of 2018, approximately 37.9 million people are infected with HIV globally.', 60.17778), ('Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.', 59.551132), ('Cholera is an infection of the small intestine by some strains of the bacterium Vibrio cholerae.', 58.748867), ('The Spanish flu, also known as the 1918 flu pandemic, was an unusually deadly influenza pandemic caused by the H1N1 influenza A virus.', 57.796642)]
